<a href="https://colab.research.google.com/github/StickMonkey615/JHCSMod4/blob/main/Final%20Classifier%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 4 Guidance

This notebook is a template for module 4b and 4c, which will be tested in Google Colab, your code needs to run there.
The structure has been provided to improve consistency and make it easier for markers to understand your code but still give students the flexibility to be creative.  You need to populate the required functions to solve this problem.  All dependencies should be documented in the next cell.

You can:
    add further cells or text blocks to extend or further explain your solution
    add further functions

Dont:
    rename functions
   

In [1]:
# Fixed dependencies - do not remove or change.
import pytest
import pandas as pd
import numpy as np
from google.colab import drive
# drive.mount('/content/gdrive/')
# Import your dependencies
!pip install --upgrade xlrd > 1.2.0
!pip install imbalanced-learn
!pip install --upgrade imbalanced-learn
import xlrd
import imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached imbalanced_learn-0.9.1-py3-none-any.whl (199 kB)


In [2]:
# Import data

def import_local_data(file_path):
    """This function needs to import the data file into collab and return a pandas dataframe
    """
    raw_df = pd.read_excel(file_path)
    return raw_df

In [3]:
local_file_path = "breast-cancer.xls"

In [4]:
# Dont change
raw_data = import_local_data(local_file_path)

### Conduct exploratory data analysis and explain your key findings - Examine the data, explain its key features and what they look like.  Highlight any fields that are anomalous.

In [5]:
# Explain your key findings

Create any data pre-processing that you will conduct on seen and unseen data.  Regardless of the model you use, this dataframe must contain only numeric features and have a strategy for any expected missing values. Any objects can that are needed to handle the test data that are dependent on the training data can be stored in the model class.  You are recommended to use sklearn Pipelines or similar functionality to ensure reproducibility.

In [6]:
class Module4_Model:
    
    def __init__(self):
        self.model = None

    def process_dataframe(sel, unproc_data):
        '''
        '''
        # Remove non-categorical data
        dm = unproc_data.pop('deg-malig')

        # Correct date types in tumor-size abd inv-nodes variables
        for i in range(0, len(unproc_data)):
            if type(unproc_data['tumor-size'][i]) is not str:
                if unproc_data['tumor-size'][i].day == 1:
                    unproc_data['tumor-size'][i] = str(unproc_data['tumor-size'][i].month) + '-' + str(unproc_data['tumor-size'][i].year-2000)
                else:
                    unproc_data['tumor-size'][i] = str(unproc_data['tumor-size'][i].day) + '-' + str(unproc_data['tumor-size'][i].month)
            if type(unproc_data['inv-nodes'][i]) is not str:
                if unproc_data['inv-nodes'][i].day == 1:
                    unproc_data['inv-nodes'][i] = str(unproc_data['inv-nodes'][i].month) + '-' + str(unproc_data['inv-nodes'][i].year-2000)
                else:
                    unproc_data['inv-nodes'][i] = str(unproc_data['inv-nodes'][i].day) + '-' + str(unproc_data['inv-nodes'][i].month)

        # Convert node-caps and irradiat variables into booleans
        for i in range(0, len(unproc_data)):
            if unproc_data['node-caps'][i] == 'yes':
                unproc_data['node-caps'][i] = True
            elif unproc_data['node-caps'][i] == 'no':
                unproc_data['node-caps'][i] = False
            else:
                unproc_data['node-caps'][i] = False
            if unproc_data['irradiat'][i] == 'yes':
                unproc_data['irradiat'][i] = True
            elif unproc_data['irradiat'][i] == 'no':
                unproc_data['irradiat'][i] = False
            else:
                unproc_data['irradiat'][i] = False


        # Encode the catagorical data (dummy variables)
        proc_data = pd.get_dummies(data=unproc_data, prefix_sep='_', drop_first=True)
    
        # Add back in non-categorical data
        proc_data.insert(0, 'deg-malig', dm)
    
        return proc_data
        
    def preprocess_training_data(self, training_df):
        """
        This function should process the training data and store any features required in the class
        """
        # Apply feature scaling
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        processed_df = sc.fit_transform(training_df)
        return processed_df

    def preprocess_test_data(self, test_df):

        # Apply feature scaling
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        processed_df = sc.fit_transform(test_df)
        return processed_df



In [7]:
# Dont change
my_model = Module4_Model()

# clean up data and create dataframe of only catagocial numeric data
clean_data = my_model.process_dataframe(raw_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarn

In [8]:
# Split your data so that you can test the effectiveness of your model
# Split the data into a Training set and a Test set
dfs = np.split(clean_data, [len(clean_data.columns)-1], axis=1)
X = dfs[0]
y = dfs[1]
#X = raw_data.iloc[:, :-1].values
#y = raw_data.iloc[:, -1].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [9]:
# Dont change
x_train_processed = my_model.preprocess_training_data(X_train)

In [10]:
# Populate preprocess_training_data and preprocess_test_data to preprocess data.
# You must process test and train separately so your model does not accidently gain information that a model wouldnt have in reality and therefore get better predictions


In [11]:
# Dont change
x_test_processed = my_model.preprocess_test_data(X_test)

In [12]:
# Create a model
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
classifier = EasyEnsembleClassifier(n_estimators=10,
                                    base_estimator=RandomForestClassifier(n_estimators=100,
                                                                          criterion='entropy',
                                                                          class_weight='balanced_subsample'))

In [13]:
# Train your model
classifier.fit(x_train_processed, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


EasyEnsembleClassifier(base_estimator=RandomForestClassifier(class_weight='balanced_subsample',
                                                             criterion='entropy'))

In [14]:
# use your model to make a prediction on unseen data
y_pred = classifier.predict(x_test_processed)

In [15]:
# Asssess the accuracy of your model and explain your key findings
# Generate confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score
cm = confusion_matrix(y_test, y_pred)
print(f"{cm}:Confusion matrix")
print("Accuracy: {:.2f} %".format(accuracy_score(y_test, y_pred)))
print("Weighted ROC AUC accuracy: {:.2f} %".format(roc_auc_score(y_test, y_pred, average='weighted')))

# Apply k-fold Cross Validation
from sklearn.model_selection import cross_val_score
from numpy import ravel
accuracies = cross_val_score(estimator = classifier, X = x_train_processed, y = ravel(y_train.values), scoring = 'roc_auc_ovo', cv = 10)
print("K-fold cross validation results")
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

[[30 15]
 [ 9 18]]:Confusion matrix
Accuracy: 0.67 %
Weighted ROC AUC accuracy: 0.67 %
K-fold cross validation results
Accuracy: 69.75 %
Standard Deviation: 9.47 %


In [16]:
# Apply Grid Search to find best model and parameters
#from sklearn.model_selection import GridSearchCV
#parameters = [{'n_estimators': [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100],
#               'criterion': ['gini', 'entropy'],
#               'class_weight': ['balanced', 'balanced_subsample', 'None']}]
#parameters = [{'n_estimators': [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100 ], 
#               'criterion': ['gini', 'entropy', 'log_loss'],
#               'class_weight': ['balanced', 'balanced_subsample'],
#               'random_state': [0]}]
#grid_search = GridSearchCV(estimator = classifier,
#                           param_grid = parameters,
#                           scoring = 'roc_auc_ovr_weighted',
#                           cv = 10,
#                           n_jobs = -1)
#grid_search.fit(x_train_processed, y_train)
#best_accuracy = grid_search.best_score_
#best_parameters = grid_search.best_params_
#print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
#print("Best Parameters:", best_parameters)

In [17]:
# Update model
#classifier = EasyEnsembleClassifier(n_estimators=best_parameters['n_estimators'],
#                                            criterion = best_parameters['criterion'],
#                                            class_weight = best_parameters['class_weight'])
#classifier = RandomForestClassifier(n_estimators=best_parameters['n_estimators'],
#                                    criterion=best_parameters['criterion'],
#                                    class_weight = 'balanced_subsample',
#                                    random_state=0)
#classifier.fit(x_train_processed, y_train)

### Unit tests:

###Checking training and test data for null values. This will work for both pd dataframes and np arrays, and ensures no null values exist.

In [18]:
def test_no_nulls(data):
    """ Assert no null values within pd dataframe or np array """
    
    # if data is numpy array, handle accordingly
    if isinstance(data, (np.ndarray)):
        assert not np.isnan(np.min(data))
    
    # if not np array, assume data is pandas dataframe
    else:
        assert data.isna().sum().sum() == 0

In [19]:
# run null data unit test on both training and test data
test_no_nulls(x_train_processed)
test_no_nulls(x_test_processed)